<a href="https://colab.research.google.com/github/arsalanpardesi/diligence_ustad/blob/main/diligence_ustad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Installing Required Libraries



In [ ]:
# Installing required libraries
!pip install datasets pandas
!pip install pymongo sentence_transformers
!pip install -U transformers
!pip install accelerate

### Data Loading and Preparation

In [ ]:
# import required libraries and import dataset

import pandas as pd
from datasets import load_dataset

In [ ]:
#Load dataset

dataset = load_dataset("apardesi/ebitda_adj_synthetic")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
#convert dataset to dataframe and explore

dataset_df = pd.DataFrame(dataset["train"])

dataset_df.sample(n=5)

,type,caption,sector,description,frequency
1,EBITDA,Non-recurring expenses,All sectors,Expenses that are not expected to recur in the...,50%
31,EBITDA,Supplier dispute costs,All sectors,Costs associated with disputes with suppliers....,25%
66,Indebtedness,Supplier loans,All sectors,Loans provided by suppliers to facilitate purc...,30%
12,EBITDA,Goodwill impairment,All sectors,"Impairment of goodwill, which may not be refle...",20%
32,Indebtedness,Debt,All sectors,"All forms of debt including loans, credit faci...",95%


In [ ]:
# Check data
# Its good to check the data further. In this case, we know that the data does not have any nulls.

print(dataset_df.isnull().sum())

type           0
caption        0
sector         0
description    0
frequency      0
dtype: int64


### Generate Embeddings

In [ ]:
# Generate embeddings for our dataset. We will generate embeddings on the description column

from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("thenlper/gte-large")

def get_embedding(text: str) -> list[float]:
    if not text.strip():
        print("The text cell is empty")
        return []

    embedding = embedding_model.encode(text)

    return embedding.tolist()


dataset_df["embedding"] = dataset_df["description"].apply(get_embedding)

dataset_df.head()

modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.9k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/670M [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

,type,caption,sector,description,frequency,embedding
0,EBITDA,Non-recurring income,All sectors,Income that is not expected to recur in the fu...,40%,"[-0.00667977798730135, 0.009119169786572456, -..."
1,EBITDA,Non-recurring expenses,All sectors,Expenses that are not expected to recur in the...,50%,"[-0.007796809077262878, -0.019644642248749733,..."
2,EBITDA,Unrealized foreign exchange losses,All sectors,Losses from changes in foreign currency values...,30%,"[-0.019023923203349113, 0.0002458317612763494,..."
3,EBITDA,Unrealized foreign exchange gains,All sectors,Gains from changes in foreign currency values ...,30%,"[-0.014858309179544449, -0.0006147018284536898..."
4,EBITDA,Stock-based compensation,All sectors,Adjustments for compensation expenses paid in ...,45%,"[0.0002475411747582257, -0.02120041847229004, ..."


### Setting up Database and vector store

In [ ]:
import pymongo
from google.colab import userdata

def get_mongo_client(mongo_uri):
    """Establish connection to the MongoDB."""
    try:
        client = pymongo.MongoClient(mongo_uri)
        print("Connection to MongoDB successful")
        return client
    except pymongo.errors.ConnectionFailure as e:
        print(f"Connection failed: {e}")
        return None

mongo_uri = userdata.get("mongo_uri")
if not mongo_uri:
    print("mongo_uri not set in environment variables")

mongo_client = get_mongo_client(mongo_uri)

# Ingest data into MongoDB
db = mongo_client["diligencedb"]
collection = db["diligence_collection"]

Connection to MongoDB successful


### Adding data to the collection

In [ ]:
#Delete any existing data, if required

collection.delete_many({})

DeleteResult({'n': 67, 'electionId': ObjectId('7fffffff0000000000000c48'), 'opTime': {'ts': Timestamp(1713117118, 110), 't': 3144}, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1713117118, 117), 'signature': {'hash': b'\x97\xe1\xd4\x934V\x894E%\xc9\t\x19\x7f\x01\x06=\xc4\xb6\x99', 'keyId': 7298798385518084098}}, 'operationTime': Timestamp(1713117118, 110)}, acknowledged=True)

In [ ]:
#Adding data to the collection

documents = dataset_df.to_dict("records")
collection.insert_many(documents)

print("Data added to MongoDB")

Data added to MongoDB


### Test Vector Search

In [ ]:
def vector_search(user_query, collection):

  #Generate embedding
  query_embedding = get_embedding(user_query)

  if query_embedding is None:
    return "Generation failed"

  #Define vector search pipeline

  pipeline = [
      {
          "$vectorSearch": {
                "index": "vector_index",
                "queryVector": query_embedding,
                "path": "embedding",
                "numCandidates": 20,
                "limit": 10,  # Return top 4 matches
          }
      },
      {
            "$project": {
                "_id": 0,  # Exclude the _id field
                "type": 1,
                "caption": 1,
                "sector" :1,
                "description":1,
                "frequency": 1,
                "score": {"$meta": "vectorSearchScore"},  # Include the search score
            }
      },
  ]

  # Perform search
  results = collection.aggregate(pipeline)
  return list(results)

In [ ]:
# Handling queries

def get_search_result(query, collection):

  get_knowledge = vector_search(query, collection)

  search_result = ""
  for result in get_knowledge:
    search_result += f"Type: {result.get('type', 'N/A')}, Caption: {result.get('caption', 'N/A')}, Sector: {result.get('sector', 'N/A')}, Description: {result.get('description', 'N/A')}, Frequency: {result.get('frequency', 'N/A')} \n"

  return search_result

In [ ]:
# Test query

query = "What are the top 3 EBITDA adjustments for technology sector and why? Please return items with Type: EBITDA only."
source_information = get_search_result(query, collection)
combined_information = f"Query: {query}\n{source_information}."

print(combined_information)

Query: What are the top 3 EBITDA adjustments for technology sector and why? Please return items with Type: EBITDA only.
Type: EBITDA, Caption: Covid-19 related adjustments, Sector: All sectors, Description: Adjustments due to the impact of Covid-19, such as government assistance programs and safety-related expenses. Counter argument by Buyer: Buyer may argue that these adjustments may set new operational standards and ongoing costs, hence should not be adjusted out. Strength: Medium., Frequency: 30% 
Type: EBITDA, Caption: Product development adjustments, Sector: Technology, Description: Adjustments for costs related to new product development. Counter argument by Buyer: Buyer may argue that these expenses reflect investment in future growth and innovation and should not be adjusted out. Strength: Weak., Frequency: 40% 
Type: EBITDA, Caption: Software development costs, Sector: Technology, Description: Adjustments for software development expenses that may be capitalized or expensed. C

In [ ]:
#type(combined_query)

###Putting an LLM on top of vector search

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it", device_map="auto")

tokenizer_config.json:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/888 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
gemma_query = "What are the top 3 EBITDA adjustments for Technology sector and explain why by each adjustment? Continue to answer using the search results below:"
combined_query = f"Query: {gemma_query}\n{source_information}."

In [ ]:
# Processing queries with Gemma
input_ids = tokenizer(combined_query, return_tensors="pt").to("cuda")
response = model.generate(**input_ids, max_new_tokens=1000)
print(tokenizer.decode(response[0]))

<bos>Query: What are the top 3 EBITDA adjustments for Technology sector and explain why by each adjustment? Continue to answer using the search results below:
Type: EBITDA, Caption: Covid-19 related adjustments, Sector: All sectors, Description: Adjustments due to the impact of Covid-19, such as government assistance programs and safety-related expenses. Counter argument by Buyer: Buyer may argue that these adjustments may set new operational standards and ongoing costs, hence should not be adjusted out. Strength: Medium., Frequency: 30% 
Type: EBITDA, Caption: Product development adjustments, Sector: Technology, Description: Adjustments for costs related to new product development. Counter argument by Buyer: Buyer may argue that these expenses reflect investment in future growth and innovation and should not be adjusted out. Strength: Weak., Frequency: 40% 
Type: EBITDA, Caption: Software development costs, Sector: Technology, Description: Adjustments for software development expenses

### Trying out Indebtedness queries

In [ ]:
# Indebtedness step 1

query1 = "What are 5 examples of Type Indebtedness with high frequency. Please return items with Type: Indebtedness"
source_information_ind = get_search_result(query1, collection)

print(source_information_ind)

Type: Indebtedness, Caption: Debt, Sector: All sectors, Description: All forms of debt including loans, credit facilities, and bonds. Counter argument by Seller: Seller may argue that debt can be refinanced or restructured to reduce burden, and thus should be discounted in its impact. Strength: Medium., Frequency: 95% 
Type: Indebtedness, Caption: Unpaid interest, Sector: All sectors, Description: Interest expenses accrued but not yet paid. Counter argument by Seller: Seller may argue that unpaid interest can be managed through refinancing or negotiating repayment terms, hence should not be overly penalized. Strength: Medium., Frequency: 30% 
Type: Indebtedness, Caption: Overdrafts, Sector: All sectors, Description: Negative balances in bank accounts that need to be repaid. Counter argument by Seller: Seller may argue that overdrafts are temporary and a part of managing cash flow efficiently, hence should not be overly penalized. Strength: Weak., Frequency: 45% 
Type: Indebtedness, Cap

In [ ]:
# Indebtedness step 2

gemma_query_ind = "Based on the search results below, explain why off-balance sheet obligations are important to consider as an Indebtedness item?"
combined_query_ind = f"Query: {gemma_query_ind}\n{source_information_ind}."

print(combined_query_ind)

Query: Based on the search results below, explain why off-balance sheet obligations are important to consider as an Indebtedness item?
Type: Indebtedness, Caption: Debt, Sector: All sectors, Description: All forms of debt including loans, credit facilities, and bonds. Counter argument by Seller: Seller may argue that debt can be refinanced or restructured to reduce burden, and thus should be discounted in its impact. Strength: Medium., Frequency: 95% 
Type: Indebtedness, Caption: Unpaid interest, Sector: All sectors, Description: Interest expenses accrued but not yet paid. Counter argument by Seller: Seller may argue that unpaid interest can be managed through refinancing or negotiating repayment terms, hence should not be overly penalized. Strength: Medium., Frequency: 30% 
Type: Indebtedness, Caption: Overdrafts, Sector: All sectors, Description: Negative balances in bank accounts that need to be repaid. Counter argument by Seller: Seller may argue that overdrafts are temporary and a

In [ ]:
# Processing queries with Gemma
input_ids = tokenizer(combined_query_ind, return_tensors="pt").to("cuda")
response = model.generate(**input_ids, max_new_tokens=1000)
print(tokenizer.decode(response[0]))

<bos>Query: Based on the search results below, explain why off-balance sheet obligations are important to consider as an Indebtedness item?
Type: Indebtedness, Caption: Debt, Sector: All sectors, Description: All forms of debt including loans, credit facilities, and bonds. Counter argument by Seller: Seller may argue that debt can be refinanced or restructured to reduce burden, and thus should be discounted in its impact. Strength: Medium., Frequency: 95% 
Type: Indebtedness, Caption: Unpaid interest, Sector: All sectors, Description: Interest expenses accrued but not yet paid. Counter argument by Seller: Seller may argue that unpaid interest can be managed through refinancing or negotiating repayment terms, hence should not be overly penalized. Strength: Medium., Frequency: 30% 
Type: Indebtedness, Caption: Overdrafts, Sector: All sectors, Description: Negative balances in bank accounts that need to be repaid. Counter argument by Seller: Seller may argue that overdrafts are temporary 